# **PREREQUISITES**

In [ ]:
# This code is from this website: https://pysource.com/2021/08/10/train-mask-r-cnn-for-image-segmentation-online-free-gpu/

%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!wget https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
!unzip Mask_RCNN_basic_1.zip
import sys
sys.path.append("/content/Mask_RCNN/mrcnn")
from m_rcnn import *
%matplotlib inline

In [ ]:
# For MQTT Protocol Functions

pip install paho-mqtt

In [ ]:
# Required libraries for the project

import cv2
import numpy as np
from mrcnn.visualize import random_colors, get_mask_contours, draw_mask
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import os
import time
import tensorflow as tf
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing import image
import paho.mqtt.client as mqttclient
import json
from skimage import io

In [ ]:
# Mount your google drive here

from google.colab import drive
drive.mount('/content/drive')

# **CHANGE CODE HERE**

The main directory should look like this:

*root_folder/*

  │

*├──  models/*

*├──  input_image/*

*├──  roi/*

*├──  masked_digits/*

*├──  cropped_digits/*

In [ ]:
# CHANGE THE VALUE OF "folder_path" TO YOUR CORRECT DIRECTORY
folder_path = "/content/drive/MyDrive/Documentation Files/(4)Step-by-step/prediction_files/"

# NO NEED TO CHANGE THESE VALUES
models_path = folder_path + "models/"                 # This directory will store the three deep learning models
input_image_path = folder_path + "input_image/"       # This directory will store the input images taken by the ESP32-CAM
roi_path = folder_path + "roi/"                       # This directory will store the ROI image file from the ROI Detection model
masked_digits_path = folder_path + "masked_digits/"   # This directory will store the masked digit image files from the Digit Detection Model
cropped_digits_path = folder_path + "cropped_digits/" # This directory will store the cropped individual image files

# **MAIN CODE**

The code below loads the models from the "*root_folder/models/*" path.

In [ ]:
test_model_ROI, inference_config_ROI = load_inference_model(1, models_path + "ROI_detection_model.h5")          # Loads the ROI Detection Model 
test_model_digits, inference_config_digits = load_inference_model(1, models_path + "digit_detection_model.h5")  # Loads the Digit Detection Model 
model_reading = models.resnext50_32x4d(pretrained=True)                                       
num_ftrs = model_reading.fc.in_features
model_reading.fc = nn.Linear(num_ftrs, 10)
checkpoint = torch.load(models_path + "digit_reading_model.pt")                                                 # Loads the Digit Reading Model
model_reading.load_state_dict(checkpoint)

In [ ]:
initial_checker = 0

while(1):
  for filename in os.listdir(input_image_path):
    print(input_image_path + filename)

  # Get the input image from the input_image_path  
  img = cv2.imread(input_image_path + filename)
  # Remove the image so that only one image will be at the directory at a time
  # You can copy the image to another directory if you wish
  os.remove(input_image_path + filename)

  cv2_imshow(img)
  # Get the ROI of the input image
  get_ROI(img)
  time.sleep(0.75)
  ROI = cv2.imread(roi_path + "ROI.png")
  cv2_imshow(ROI)

  # Get the individual image files from the ROI
  get_digits(ROI)

  # Compose the string of the actual reading of the water meter
  actual_reading = ""

  # Iterate throught the cropped_digits_path and predict each image
  for filename in os.listdir(cropped_digits_path):
    image = cv2.imread(cropped_digits_path + filename)
    cv2_imshow(image)
    model_reading.eval()
    with torch.no_grad():
      prediction = predict_image(cropped_digits_path + filename)
      print("Predicted Class: ",prediction)
    actual_reading += str(prediction)
    os.remove(cropped_digits_path + filename)


  print("Predicted watermeter reading is " + str(int(actual_reading)))

  # Check if the image is the intitial image and set the values of the bill accordingly
  if(initial_checker == 0):
    initial_reading = int(actual_reading)
    final_reading = int(actual_reading)
    basic_charge = 0
    FCDA = 0
    environmental_charge = 0
    sewer_charge = 0
    vat = 0 
    before_tax = 0
    total_bill = 0
    initial_checker = 1

  else:
    final_reading = int(actual_reading)

  # Get the bill details using these variables
  consumption, basic_charge, FCDA, environmental_charge, sewer_charge, vat, before_tax, total_bill = get_bill()
  # Set up the MQTT_MSG using json.dumps() containing the bill details
  MQTT_MSG = json.dumps({"payload_fields": {"reading": int(actual_reading),
                                            "basic charge": int(basic_charge),
                                            "FCDA": int(FCDA),
                                            "Environmental Charge": int(environmental_charge),
                                            "Sewer Charge": int(sewer_charge),
                                            "VAT Charge": int(vat),
                                            "Before TAX": int(before_tax),
                                            "bill": int(total_bill)
                                            }});
  # Connect to the MQTT Broker                                          
  MQTT_connect(MQTT_MSG)

  # Set the time you want to wait. In this case it is a 30 second wait before the next reading
  time.sleep(30) 

# **FUNCTIONS**

`get_ROI(img)` gets the ROI from an input image using the ROI Detection Model

Once the ROI is obtained, the ROI image file in png will be saved to the "*root_folder/roi/*" path as "*ROI.png*"

**Parameters**

-`img` - image file from `cv2.imread()` function (This will be  the image of the whole water meter)


In [ ]:
def get_ROI(img): # GETS THE ROI USING ROI_detection_model.h5 
  global masked_image
  try:
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  except:
    print("error")
    total_counter -= 1
  else:
  
    # Detect results
    r = test_model_ROI.detect([image])[0]

    object_count = len(r["class_ids"])
    colors = random_colors(50)
    for i in range(object_count):
        # 1. Mask
        mask = r["masks"][:, :, i]
        contours = get_mask_contours(mask)
        for cnt in contours:
            mask = np.zeros(img.shape, dtype=np.uint8)
            cv2.fillPoly(mask, pts=[cnt], color=(255, 255, 255))
            masked_image = cv2.bitwise_and(img, mask)

    # Make mask transparent
    try:
      tmp = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    except:
      print("error")
    else:
      _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
      b, g, r = cv2.split(masked_image)
      rgba = [b,g,r, alpha]
      masked_tr = cv2.merge(rgba,4)
      idx = np.where(masked_tr[: ,: , 3] > 0)
      x0, y0, x1, y1 = idx[1].min(), idx[0].min(), idx[1].max(), idx[0].max()
      out = Image.fromarray(masked_tr[y0: y1 + 1, x0: x1 + 1,: ])
    
      out.save(roi_path + "ROI.png")

`get_digits(img)` almost does the same thing as `get_ROI(img)` except this one extends its functionality by doing the process multiple times to get the masks of each individual digit in the ROI.


Once the each masked digit is obtained, the individual image files in png will be saved to the "*root_folder/masked_digits/*" path. Furthermore, the masked digits will be cropped and stored to the "*root_folder/cropped_digits/*" path.

**Parameters**

-`img` - ROI image file from `get_ROI(img)` function (This will be read from the the "*root_folder/roi/*" folder using `cv2.imread()`) 


In [ ]:
def get_digits(img): # EXTRACTS THE DIGTS FROM THE MASKED DIGIT IMAGES
  global masked_image
  global number
  try:
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  except:
    print("error")
    total_counter -= 1
  else:
    coordinates = []

    # Detect results
    r = test_model_digits.detect([image])[0]

    object_count = len(r["class_ids"])
    colors = random_colors(50)
    for i in range(object_count):
        # 1. Mask
        mask = r["masks"][:, :, i]
        contours = get_mask_contours(mask)
        #print(type(mask))
        for cnt in contours:
            #cv2.polylines(img, [cnt], True, colors[i], 2)
            mask = np.zeros(img.shape, dtype=np.uint8)
            cv2.fillPoly(mask, pts=[cnt], color=(255, 255, 255))
            img_w_mask = draw_mask(img, [cnt], colors[i])
            masked_image = cv2.bitwise_and(img, mask)
            x_coor = find_nearest_white(mask, TARGET)
            coordinates.append(int(x_coor[1]))
        cv2.imwrite(masked_digits_path + str(x_coor[1]) + '.png', masked_image) 

    number = 0
    for filename in sorted(coordinates):
        masked_image = cv2.imread(masked_digits_path + str(filename) + ".png")
        #cv2_imshow(masked_image)
        #print(filename)
        try:
          tmp = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
        except:
          print("error")
        else:
          _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
          b, g, r = cv2.split(masked_image)
          rgba = [b,g,r, alpha]
          masked_tr = cv2.merge(rgba,4)

          idx = np.where(masked_tr[: ,: , 3] > 0)

          x0, y0, x1, y1 = idx[1].min(), idx[0].min(), idx[1].max(), idx[0].max()
 
          out = Image.fromarray(masked_tr[y0: y1 + 1, x0: x1 + 1,: ])

          out.save(cropped_digits_path + 'img({}).png'.format(number))
          #out.save('/content/drive/MyDrive/Results/img({}).png'.format(number))
          number = number + 1
          os.remove(masked_digits_path + str(filename) + ".png")


`find_nearest_white(img, target) ` finds the coordinates of the image with respect to the x axis. This will serve as indicators for the ordering of the digits from left to right for the `get_digits(img)` function.

**Parameters**

-`img` - the masked digit image file inside the `get_digits(img)` function

-`target` - a non-zero pixel in the image

**Returns**

-coordinates of a corresponding nonzero pixel

In [ ]:
TARGET = (255,255) # ORDERS THE DIGITS CORRECTLY
def find_nearest_white(img, target):

    nonzero = np.argwhere(img == 255)
    distances = np.sqrt((nonzero[:,0] - TARGET[0]) ** 2 + (nonzero[:,1] - TARGET[1]) ** 2)
    nearest_index = np.argmin(distances)
    return nonzero[nearest_index]

`predict_image(image_path)` is a function that returns the index of the most likely value of the digit image file based on a class label from 0 to 9.

**Parameters**

-`image_path` - path of the input image 


**Returns**

-index of the class label which is from 0 to 9.

In [ ]:
def predict_image(image_path): # PREDICTS THE IMAGE USING digit_reading_model.pt
  transformation = transforms.Compose([
      transforms.ToPILImage(),                                 
      transforms.Resize((32,32)),
      #transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])
  image_tensor = transformation(image).float()
  image_tensor = image_tensor.unsqueeze_(0)

  if torch.cuda.is_available():
      image_tensor.cuda()

  #input = Variable(image_tensor)
  output = model_reading(image_tensor)
  #print(output)
  index = output.data.numpy().argmax()
  return index

`MQTT_connect(MQTT_MSG)` is a function that connects to a MQTT broker

**Parameters**

-`MQTT_MSG` - json string from a Python object using `json.dumps()`. This will contain the values that you want to display on the MQTT broker.


In [ ]:
def MQTT_connect(MQTT_MSG): # ESTABLISHES CONNECTION TO THE MQTT BROKER (things.ph)
  def on_connect(client, usedata, flags, rc):
    if rc == 0:
      #print("Client is connected")
      global connected
      connected = True
    else:
      print("Connection failed.")


  connected = False
  
  # EDIT THE VALUES TO CORRECTLY CONNECT TO THE MQTT BROKER YOU ARE USING
  broker_address = "mqtt.things.ph"
  port = 1883
  user = "61b057b73bc1a3388183e64c"
  password = "Kfq3BA5tH3zCVkXpP3mcFNlI"
  ##############

  client = mqttclient.Client("MQTT")
  client.username_pw_set(user, password=password)
  client.on_connect = on_connect
  client.connect(broker_address, port=port)
  client.loop_start()
  #while connected != True:
  #  time.sleep(0.2)
  client.publish("mqtt/watermeter", MQTT_MSG)

`get_bill()` function calculates the estimated bill using the *initial_reading* and *final_reading* values.


**Returns**

-the bill breakdown such as *basic_charge, FCDA, environmental_charge, sewer_charge, vat, before_tax, total_bill*

In [ ]:
def get_bill(): # CALCULATES THE ESTIMATED BILL

  # MANILA WATER
  # bill_lessthan10 = 63.16
  # bill_first10 = 111.27
  # bill_next10 = 13.56
  # bill_next20_1 = 25.71
  # bill_next20_2 = 33.89
  # bill_next20_3 = 39.58
  # bill_next20_4 = 41.49
  # bill_next50_1 = 43.34
  # bill_next50_2 = 45.2
  # bill_over200 = 47.06

  # MAYNILAD
  bill_lessthan10 = 96.32
  bill_first10 = 164.16
  bill_next10 = 20.03
  bill_next20_1 = 38.09
  bill_next20_2 = 50.03
  bill_next20_3 = 58.45
  bill_next20_4 = 61.13
  bill_next50_1 = 63.93
  bill_next50_2 = 66.78
  bill_over200 = 69.60

  consumption = final_reading - initial_reading
  consumption_ctr = 10
  counter = 10
  expectedBill = bill_first10
  if(consumption < 10):
    expectedBill = bill_lessthan10
  elif(consumption == 10):
    expectedBill = bill_first10
  else:
    expectedBill = bill_first10
    while(counter < consumption):
      if(consumption_ctr < 20): #next 10
        expectedBill += bill_next10
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 40): #next20 1
        expectedBill += bill_next20_1
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 60): #next20 2
        expectedBill += bill_next20_2
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 80): #next20 3
        expectedBill += bill_next20_3
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 100): #next20 4
        expectedBill += bill_next20_4
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 150): #next50 1
        expectedBill += bill_next50_1
        counter += 1
        consumption_ctr += 1
      elif(consumption_ctr < 200): #next50 2
        expectedBill += bill_next50_2
        counter += 1
        consumption_ctr += 1
      else:
        expectedBill += bill_over200
        counter += 1
        consumption_ctr += 1

  basic_charge = expectedBill
  FCDA = 0.0055*basic_charge
  environmental_charge = 0.2*(basic_charge - FCDA)
  sewer_charge = 0
  vat = 0.12*(basic_charge-FCDA+environmental_charge+sewer_charge)
  before_tax = basic_charge-FCDA+environmental_charge+sewer_charge
  total_bill = basic_charge-FCDA+environmental_charge+sewer_charge+vat
  print("Water Consumption = " + str(consumption) + " cubic meters")
  print("Basic Charge = ",basic_charge)
  print("FCDA =", FCDA)
  print("Environmental Charge =", environmental_charge)
  print("Sewer Charge =", sewer_charge)
  print("ADD VAT =", vat)
  print("Total Current Charge Before Tax =", before_tax)
  print("TOTAL BILL =", total_bill)
  return consumption, basic_charge, FCDA, environmental_charge, sewer_charge, vat, before_tax, total_bill